In [ ]:
import itertools
from collections import Counter

import numpy as np
import matplotlib.pyplot as plt
from skimage import data
from skimage.util import view_as_blocks
from skimage.color import rgb2gray
from skimage.metrics import peak_signal_noise_ratio as psnr, structural_similarity as ssim
from scipy.fftpack import dct, idct

import rle
import huffman

In [ ]:
block_shape = (8, 8)

chelsea = data.chelsea()
chelsea = np.rint(rgb2gray(chelsea) * 255).astype(np.uint8)

height = chelsea.shape[0] // block_shape[0] * block_shape[0]
width = chelsea.shape[1] // block_shape[1] * block_shape[1]

chelsea = chelsea[:height, :width]

In [ ]:
plt.imshow(chelsea, cmap='gray')

In [ ]:
def filesize(array):
    kb = 1024
    
    bytes_count = len(bytearray(array))
    return str(bytes_count // kb) + ' kb'

In [ ]:
filesize(chelsea)

In [ ]:
def dct2d(array):
    return dct(dct(array, norm='ortho', axis=-1), norm='ortho', axis=-2)

def idct2d(array):
    return idct(idct(array, norm='ortho', axis=-1), norm='ortho', axis=-2)

In [ ]:
quantization_table = np.array([
    [16,  11,  10,  16,  24,   40,   51,   61 ],
    [12,  12,  14,  19,  26,   58,   60,   55 ],
    [14,  13,  16,  24,  40,   57,   69,   56 ],
    [14,  17,  22,  29,  51,   87,   80,   62 ],
    [18,  22,  37,  56,  68,   109,  103,  77 ],
    [24,  35,  55,  64,  81,   104,  113,  92 ],
    [49,  64,  78,  87,  103,  121,  120,  101],
    [72,  92,  95,  98,  112,  100,  103,  99 ]])

In [ ]:
def quantize_block(block):
    coefs = dct2d(block) / quantization_table
    
    return np.rint(coefs + 127).clip(0, 255).astype(np.uint8)

def restore_block(block):
    return np.rint(idct2d((block.astype(np.int8) - 127) * quantization_table)).clip(0, 255).astype(np.uint8)

In [ ]:
# TODO : zeros instead 127 ??

In [ ]:
compressed = chelsea.copy()
view = view_as_blocks(compressed, block_shape)

In [ ]:
coefs = np.zeros_like(view, dtype=np.uint8)

for row in range(view.shape[0]):
    for col in range(view.shape[1]):
        coefs[row][col] = quantize_block(view[row][col])

In [ ]:
zigzag = np.array(
    [[0,  1,  5,  6,  14, 15, 27, 28],
    [2,  4,  7,  13, 16, 26, 29, 42],
    [3,  8,  12, 17, 25, 30, 41, 43],
    [9,  11, 18, 24, 31, 40, 44,53],
    [10, 19, 23, 32, 39, 45, 52,54],
    [20, 22, 33, 38, 46, 51, 55,60],
    [21, 34, 37, 47, 50, 56, 59,61],
    [35, 36, 48, 49, 57, 58, 62,63]])

In [ ]:
compressed_data = rle.code(coefs.flatten())  # TODO: zigzag instead flatten

In [ ]:
filesize(compressed_data)

In [ ]:
codes = huffman.codes(Counter(compressed_data))

In [ ]:
length, coded_compressed_data = huffman.code(compressed_data, codes)

In [ ]:
filesize(coded_compressed_data)

In [ ]:
decoded_compressed_data = huffman.decode(coded_compressed_data, codes, length)

In [ ]:
view = np.array(rle.decode(decoded_compressed_data)).reshape(height // block_shape[0], width // block_shape[1], *block_shape)

In [ ]:
decompressed = np.zeros((height, width), dtype=np.uint8)
decompressed_view = view_as_blocks(decompressed, block_shape)

for row in range(view.shape[0]):
    for col in range(view.shape[1]):
        decompressed_view[row][col] = restore_block(view[row][col])

In [ ]:
plt.imshow(decompressed, cmap='gray')

In [ ]:
psnr(chelsea, decompressed)

In [ ]:
ssim(chelsea, decompressed)